In [1]:
import pandas as pd
import os

# Load raw dataset
df = pd.read_excel("../data/raw/marketing_campaign.xlsx")

# Drop duplicates
df.drop_duplicates(inplace=True)

# Identify numeric columns (excluding IDs or postal codes)
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
id_like_cols = [col for col in numeric_cols if 'id' in col.lower() or 'zip' in col.lower()]
numeric_cols = [col for col in numeric_cols if col not in id_like_cols]

# Fill missing numeric values with median
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Identify date columns (currently only Dt_Customer, but scalable)
date_cols = [col for col in df.columns if "date" in col.lower() or "dt_" in col.lower()]
if 'Dt_Customer' in df.columns:
    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce')

# Identify categorical columns (exclude dates)
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols = [col for col in cat_cols if col not in date_cols]

# Fill missing categorical values
df[cat_cols] = df[cat_cols].fillna("Unknown")

# Convert categorical columns to 'category' type
for col in cat_cols:
    df[col] = df[col].astype('category')

df_fe = df.copy()

# 1. Age
current_year = pd.Timestamp.today().year
df_fe['Age'] = current_year - df_fe['Year_Birth']

# 2. Total kids
df_fe['Total_Kids'] = df_fe['Kidhome'] + df_fe['Teenhome']

# 3. Total spending on products
product_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 
                'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df_fe['Total_Spent'] = df_fe[product_cols].sum(axis=1)

# 4. Total purchases & average spend
purchase_cols = ['NumDealsPurchases', 'NumWebPurchases', 
                 'NumCatalogPurchases', 'NumStorePurchases']
df_fe['Total_Purchases'] = df_fe[purchase_cols].sum(axis=1)

# Replace division-by-zero NaN with median instead of 0
df_fe['Avg_Spent_per_Purchase'] = (
    df_fe['Total_Spent'] / df_fe['Total_Purchases'].replace(0, pd.NA)
)
df_fe['Avg_Spent_per_Purchase'] = df_fe['Avg_Spent_per_Purchase'].fillna(
    df_fe['Avg_Spent_per_Purchase'].median()
)

# 5. Customer tenure (in days)
if 'Dt_Customer' in df_fe.columns:
    df_fe['Customer_Tenure_Days'] = (
        pd.Timestamp.today() - df_fe['Dt_Customer']
    ).dt.days

# 6. Total campaign acceptance
campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 
                 'AcceptedCmp4', 'AcceptedCmp5', 'Response']
df_fe['Total_Campaign_Accepted'] = df_fe[campaign_cols].sum(axis=1)

# 7. High spender flag (top 25% spender)
spend_threshold = df_fe['Total_Spent'].quantile(0.75)
df_fe['High_Spender'] = (df_fe['Total_Spent'] > spend_threshold).astype(int)

# 8. Family status segmentation
df_fe['Family_Status'] = df_fe['Marital_Status'].astype(str) + "_" + df_fe['Total_Kids'].astype(str)

# 9. Spending ratios per category
for col in product_cols:
    df_fe[col + '_Ratio'] = (
        df_fe[col] / df_fe['Total_Spent'].replace(0, pd.NA)
    ).fillna(0)

os.makedirs("../data/processed", exist_ok=True)
save_path = "../data/processed/marketing_campaign_feature_engineered.xlsx"
df_fe.to_excel(save_path, index=False)

print("✅ Preprocessing + Feature engineering complete!")
print(f"📁 File saved at: {save_path}")


C:\Users\vemul\AppData\Local\Temp\ipykernel_2304\3841389488.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_fe['Avg_Spent_per_Purchase'] = df_fe['Avg_Spent_per_Purchase'].fillna(


✅ Preprocessing + Feature engineering complete!
📁 File saved at: ../data/processed/marketing_campaign_feature_engineered.xlsx
